## Step 1: Data Preprocessing

In [1]:
import os
import json
import spacy
import string
from tqdm import tqdm

# Load spacy tokenizer
nlp = spacy.load("en_core_web_sm")

# Paths
ANNOTATION_FILE = "annotations/captions_train2017.json"
IMAGE_FOLDER = "train2017"
OUTPUT_JSON = "processed_captions.json"

# Load JSON
with open(ANNOTATION_FILE, 'r') as f:
    data = json.load(f)

# Mapping: image_id → [captions]
id_to_captions = {}
for annot in tqdm(data['annotations']):
    image_id = annot['image_id']
    caption = annot['caption'].lower().strip()

    # Remove punctuation
    caption = caption.translate(str.maketrans('', '', string.punctuation))

    # Tokenize using spaCy
    tokens = [token.text for token in nlp(caption) if not token.is_space]

    # Add start and end tokens
    cleaned_caption = "startseq " + " ".join(tokens) + " endseq"

    str_image_id = f"{image_id:012d}"  # Ensure zero-padded 12-digit string
    if str_image_id not in id_to_captions:
        id_to_captions[str_image_id] = []
    id_to_captions[str_image_id].append(cleaned_caption)

# Build mapping: image_id → full path + captions
final_data = []
for image_id, captions in id_to_captions.items():
    filename = f"{image_id}.jpg"
    path = os.path.join(IMAGE_FOLDER, filename)
    if os.path.exists(path):
        final_data.append({
            "image_id": image_id,
            "file_path": path,
            "captions": captions
        })

# Save preprocessed data
with open(OUTPUT_JSON, "w") as f:
    json.dump(final_data, f, indent=2)

print(f"Saved {len(final_data)} image-caption entries to {OUTPUT_JSON}")



100%|██████████| 591753/591753 [40:11<00:00, 245.35it/s]


Saved 118287 image-caption entries to processed_captions.json


In [ ]:
# Resizing Images

import os
import json
from PIL import Image
from tqdm import tqdm

# Constants
INPUT_JSON = "processed_captions.json"
RESIZED_DIR = "resized_train2017"
TARGET_SIZE = (224, 224)

# Handle resampling based on Pillow version
try:
    resample_mode = Image.Resampling.LANCZOS
except AttributeError:
    resample_mode = Image.LANCZOS  # Pillow <10

# Create output directory if not exists
os.makedirs(RESIZED_DIR, exist_ok=True)

# Load image paths from JSON
with open(INPUT_JSON, "r") as f:
    data = json.load(f)

saved_count = 0

# Resize loop
for entry in tqdm(data, desc="Resizing images"):
    original_path = entry["file_path"]
    filename = os.path.basename(original_path)
    resized_path = os.path.join(RESIZED_DIR, filename)

    try:
        if not os.path.exists(original_path):
            print(f"Not found: {original_path}")
            continue

        img = Image.open(original_path).convert("RGB")
        img = img.resize(TARGET_SIZE, resample=resample_mode)
        img.save(resized_path)
        saved_count += 1

    except Exception as e:
        print(f"Failed: {original_path} → {e}")

print(f"\nDone: {saved_count} images saved in '{RESIZED_DIR}'")


Resizing images: 100%|██████████| 118287/118287 [12:47<00:00, 154.18it/s]


Done: 118287 images saved in 'resized_train2017'


In [2]:
import os
import json
import spacy
import string
from tqdm import tqdm

# Load tokenizer
nlp = spacy.load("en_core_web_sm")

# Paths
ANNOTATION_FILE = "annotations/captions_val2017.json"
IMAGE_FOLDER = "val2017"
OUTPUT_JSON = "processed_val_captions.json"

# Load annotations
with open(ANNOTATION_FILE, 'r') as f:
    data = json.load(f)

# Create mapping
id_to_captions = {}
for annot in tqdm(data['annotations']):
    image_id = annot['image_id']
    caption = annot['caption'].lower().strip()

    # Remove punctuation
    caption = caption.translate(str.maketrans('', '', string.punctuation))

    # Tokenize with spaCy
    tokens = [token.text for token in nlp(caption) if not token.is_space]

    # Add start and end tokens
    cleaned_caption = "startseq " + " ".join(tokens) + " endseq"

    str_image_id = f"{image_id:012d}"  # Convert to 12-digit string
    if str_image_id not in id_to_captions:
        id_to_captions[str_image_id] = []
    id_to_captions[str_image_id].append(cleaned_caption)

# Build final list
final_data = []
for image_id, captions in id_to_captions.items():
    filename = f"{image_id}.jpg"
    path = os.path.join(IMAGE_FOLDER, filename)
    if os.path.exists(path):
        final_data.append({
            "image_id": image_id,
            "file_path": path,
            "captions": captions
        })

# Save
with open(OUTPUT_JSON, "w") as f:
    json.dump(final_data, f, indent=2)

print(f"Saved {len(final_data)} validation image-caption pairs to {OUTPUT_JSON}")


100%|██████████| 25014/25014 [01:44<00:00, 239.08it/s]


Saved 5000 validation image-caption pairs to processed_val_captions.json


In [7]:
# Processed Val Images

import os
import json
from PIL import Image
from tqdm import tqdm

# Constants

INPUT_JSON = "processed_val_captions.json"
RESIZED_DIR = "resized_val2017"

TARGET_SIZE = (224, 224)

# Handle resampling based on Pillow version
try:
    resample_mode = Image.Resampling.LANCZOS
except AttributeError:
    resample_mode = Image.LANCZOS  # Pillow <10

# Create output directory if not exists
os.makedirs(RESIZED_DIR, exist_ok=True)

# Load image paths from JSON
with open(INPUT_JSON, "r") as f:
    data = json.load(f)

saved_count = 0

# Resize loop
for entry in tqdm(data, desc="Resizing images"):
    original_path = entry["file_path"]
    filename = os.path.basename(original_path)
    resized_path = os.path.join(RESIZED_DIR, filename)

    try:
        if not os.path.exists(original_path):
            print(f"⚠️  Not found: {original_path}")
            continue

        img = Image.open(original_path).convert("RGB")
        img = img.resize(TARGET_SIZE, resample=resample_mode)
        img.save(resized_path)
        saved_count += 1

    except Exception as e:
        print(f"Failed: {original_path} → {e}")

print(f"\nDone: {saved_count} images saved in '{RESIZED_DIR}'")

Resizing images: 100%|██████████| 5000/5000 [00:31<00:00, 157.92it/s]


Done: 5000 images saved in 'resized_val2017'


## Step 2: Tokenizing and Vocabulary Setup

In [35]:
import json
from keras.preprocessing.text import Tokenizer
import pickle

# Set a maximum vocabulary size to filter rare words
MAX_VOCAB_SIZE = 35000  # or 5000 for stricter filtering

# Load processed captions first
with open("processed_captions.json", "r") as f:
    train_data = json.load(f)
with open("processed_val_captions.json", "r") as f:
    val_data = json.load(f)

# Extract all captions (they already contain 'startseq' and 'endseq')
all_captions = []
for entry in train_data + val_data:
    all_captions.extend(entry["captions"])

# Initialize and fit tokenizer with vocab size limit
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<unk>", filters='', lower=True)
tokenizer.fit_on_texts(all_captions)

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print(f"Tokenizer created with vocab size limit: {MAX_VOCAB_SIZE}")
print(f"Actual vocab used (word_index length): {len(tokenizer.word_index)}")


Tokenizer created with vocab size limit: 35000
Actual vocab used (word_index length): 29586


In [36]:
# Sample texts with common and rare words
texts = [
    "a man is sitting on a bench",            # Common words (likely in top 10,000)
    "flibbertigibbet is dancing joyfully",    # Rare words (likely outside top 10,000)
    "startseq a cat is playing endseq"         # Contains special tokens and common words
]

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(texts)

# Print the sequences
for text, seq in zip(texts, sequences):
    print(f"Text: {text}")
    print(f"Sequence: {seq}\n")

Text: a man is sitting on a bench
Sequence: [2, 12, 11, 14, 5, 2, 100]

Text: flibbertigibbet is dancing joyfully
Sequence: [1, 11, 2371, 9948]

Text: startseq a cat is playing endseq
Sequence: [3, 2, 49, 11, 55, 4]



In [37]:
import pickle

def save_tokenizer(tokenizer, file_path="tokenizer.pkl"):
    with open(file_path, "wb") as f:
        pickle.dump(tokenizer, f)
    print(f"Tokenizer saved to {file_path}")


In [38]:
import pickle

def load_tokenizer(file_path="tokenizer.pkl"):
    with open(file_path, "rb") as f:
        tokenizer = pickle.load(f)
    print(f"Tokenizer loaded from {file_path}")
    return tokenizer


In [39]:
# After fitting the tokenizer
save_tokenizer(tokenizer)

# Later (e.g. evaluation or inference)
tokenizer = load_tokenizer()

Tokenizer saved to tokenizer.pkl
Tokenizer loaded from tokenizer.pkl


## Step 3: Image Feature Extraction

In [56]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import pickle
import json

# Paths
train_json = "processed_captions.json"
image_dir = "train2017"  # same as used in preprocessing
feature_save_path = "train_image_features_tfhub.pkl"
image_size = (224, 224)

# Load MobileNetV2 feature extractor from TF Hub
mobilenet_v2_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(mobilenet_v2_url, input_shape=(224, 224, 3), trainable=False)

# Load only relevant training image paths from JSON
with open(train_json, "r") as f:
    train_data = json.load(f)

def extract_features(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0,1]
    img_array = np.expand_dims(img_array, axis=0)
    features = feature_extractor(img_array)
    features = tf.squeeze(features)
    return features.numpy()

# Extract features only for images listed in JSON
image_features = {}
for entry in tqdm(train_data):
    image_id = str(entry["image_id"]).zfill(12)
    image_path = entry["file_path"]
    try:
        feats = extract_features(image_path)
        image_features[image_id] = feats
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Save features
with open(feature_save_path, "wb") as f:
    pickle.dump(image_features, f)

print(f"Saved {len(image_features)} training image features to {feature_save_path}")


100%|██████████| 118287/118287 [1:20:23<00:00, 24.52it/s]


Saved 118287 training image features to train_image_features_tfhub.pkl


In [57]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import pickle
import json

# Paths
val_json = "processed_val_captions.json"
val_image_features_path = "val_image_features_tfhub.pkl"
image_size = (224, 224)

# Load MobileNetV2 from TF Hub
mobilenet_v2_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(mobilenet_v2_url, input_shape=(224, 224, 3), trainable=False)

# Load image metadata from JSON
with open(val_json, "r") as f:
    val_data = json.load(f)

# Feature extraction function
def extract_features(image_path):
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = feature_extractor(img_array)
    return tf.squeeze(features).numpy()

# Extract features from listed images only
val_image_features = {}
for entry in tqdm(val_data):
    image_id = str(entry["image_id"]).zfill(12)
    image_path = entry["file_path"]
    try:
        feats = extract_features(image_path)
        val_image_features[image_id] = feats
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Save features
with open(val_image_features_path, "wb") as f:
    pickle.dump(val_image_features, f)

print(f"Saved {len(val_image_features)} validation features to {val_image_features_path}")


100%|██████████| 5000/5000 [03:33<00:00, 23.46it/s]

Saved 5000 validation features to val_image_features_tfhub.pkl


## Step 4: Captioning Training Data

In [ ]:
train_image_features = {str(int(k)).zfill(12): v for k, v in train_image_features_tfhub.items()}

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

class DataGenerator(Sequence):
    def __init__(self, image_features, captions_dict, tokenizer, max_len, batch_size=64, vocab_size=None):
        self.image_features = {
            str(int(k)).zfill(12): v for k, v in image_features.items()
        }  # Normalize keys
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.batch_size = batch_size
        self.vocab_size = vocab_size or (len(tokenizer.word_index) + 1)

        # Normalize caption image IDs as well
        self.captions_dict = {
            str(int(k)).zfill(12): v for k, v in captions_dict.items()
        }

        # Prepare all image IDs
        self.image_ids = list(self.captions_dict.keys())
        self.indexes = np.arange(len(self.image_ids))
        self.on_epoch_end()

        # Preprocess captions into (image_id, in_seq, out_seq) format
        self.data = []
        for img_id in self.image_ids:
            captions = self.captions_dict[img_id]
            for cap in captions:
                seq = tokenizer.texts_to_sequences([cap])[0]
                for i in range(1, len(seq)):
                    in_seq = seq[:i]
                    out_seq = seq[i]
                    self.data.append((img_id, in_seq, out_seq))

    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self, idx):
        batch_data = self.data[idx * self.batch_size: (idx + 1) * self.batch_size]

        batch_image_features = []
        batch_in_seqs = []
        batch_out_seqs = []

        for img_id, in_seq, out_seq in batch_data:
            features = self.image_features.get(img_id)
            if features is None:
                continue
            batch_image_features.append(features)
            batch_in_seqs.append(in_seq)
            batch_out_seqs.append(out_seq)

        # If no data found, return empty arrays with correct shapes
        if len(batch_image_features) == 0:
            print(f"Warning: Empty batch at index {idx}")
            return [np.empty((0, 1280)), np.empty((0, self.max_len))], np.empty((0, self.vocab_size))

        # Pad sequences and one-hot encode output
        batch_in_seqs = pad_sequences(batch_in_seqs, maxlen=self.max_len, padding='post')
        batch_out_seqs = to_categorical(batch_out_seqs, num_classes=self.vocab_size)
        batch_image_features = np.array(batch_image_features)

        return [batch_image_features, batch_in_seqs], batch_out_seqs

    def on_epoch_end(self):
        np.random.shuffle(self.indexes)


In [ ]:
train_generator = DataGenerator(train_image_features, train_captions, tokenizer, max_len)

# Safely get first non-empty batch
for i in range(len(train_generator)):
    batch = train_generator[i]
    if batch[0][0].shape[0] > 0:
        print("Input Image Features Shape:", batch[0][0].shape)
        print("Input Sequences Shape:", batch[0][1].shape)
        print("Output Shape:", batch[1].shape)
        break


In [11]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def beam_search_caption(model, tokenizer, image_feature, max_len, beam_width=3):
    start_token = tokenizer.word_index['startseq']
    end_token = tokenizer.word_index['endseq']
    
    sequences = [[list([start_token]), 0.0]]  # (sequence, score)

    for _ in range(max_len):
        all_candidates = []
        for seq, score in sequences:
            if seq[-1] == end_token:
                all_candidates.append((seq, score))
                continue

            seq_input = pad_sequences([seq], maxlen=max_len, padding='post')

            preds = model.predict([image_feature, seq_input], verbose=0)

            # Handle if output is (1, vocab_size) or (1, max_len, vocab_size)
            if preds.ndim == 3:
                preds = preds[0, -1, :]
            else:
                preds = preds[0]

            top_indices = preds.argsort()[-beam_width:][::-1]
            for idx in top_indices:
                # Anti-repetition: skip if token repeats 3 times
                if len(seq) >= 2 and seq[-1] == seq[-2] == idx:
                    continue
                candidate = seq + [idx]
                candidate_score = score - np.log(preds[idx] + 1e-10)
                all_candidates.append((candidate, candidate_score))

        sequences = sorted(all_candidates, key=lambda tup: tup[1])[:beam_width]

        if all(seq[-1] == end_token for seq, _ in sequences):
            break

    best_seq = sequences[0][0]
    if end_token in best_seq:
        best_seq = best_seq[:best_seq.index(end_token)]

    inv_map = {v: k for k, v in tokenizer.word_index.items()}
    caption_words = [inv_map.get(idx, '') for idx in best_seq if idx not in [start_token, end_token]]
    
    return ' '.join(caption_words)


In [ ]:
# Define the image ID
image_id = '000000000285'

# Extract and reshape image feature to match model input
image_feature = val_image_features.get(image_id)
if image_feature is None:
    raise ValueError(f"No image feature found for ID: {image_id}")
image_feature = np.expand_dims(image_feature, axis=0)  # Shape: (1, feature_dim)

# Generate caption using beam search
caption = beam_search_caption(model, tokenizer, image_feature, max_len=49, beam_width=5)
print("Generated Caption:", caption)

In [37]:
print(list(val_image_features.keys())[:10])

['000000000139', '000000000285', '000000000632', '000000000724', '000000000776', '000000000785', '000000000802', '000000000872', '000000000885', '000000001000']


In [81]:
print("Vocabulary size:", len(tokenizer.word_index))
print("Example word from index:", tokenizer.index_word.get(20, 'Not found'))

Vocabulary size: 29586
Example word from index: are


In [5]:
from pathlib import Path
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, preprocessing
from nltk.translate.bleu_score import corpus_bleu
from typing import List, Dict, Tuple

In [6]:
def create_sequences(tokenizer: preprocessing.text.Tokenizer,
                     max_len: int,
                     desc_list: List[str],
                     photo_feature: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Generate (<img feat>, <partial seq>) → <next word> tuples for one image."""
    vocab_size = len(tokenizer.word_index) + 1
    X1, X2, y = [], [], []
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = preprocessing.sequence.pad_sequences([in_seq], maxlen=max_len, padding='post')[0]
            out_seq = tf.keras.utils.to_categorical([out_seq], num_classes=vocab_size)[0]
            X1.append(photo_feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def create_sequences(tokenizer, max_len, captions, photo_feat):
    X1, X2, y = [], [], []
    vocab_size = len(tokenizer.word_index) + 1

    photo_feat = np.array(photo_feat, dtype=np.float32)  # ensure dtype

    for caption in captions:
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
            in_seq, out_word = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_len)[0]

            out_seq = np.zeros(vocab_size, dtype=np.float32)
            out_seq[out_word] = 1.0

            X1.append(photo_feat)
            X2.append(in_seq)
            y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y)




In [8]:
import tensorflow as tf

def data_generator(features, descriptions, tokenizer, max_len):
    for img_id, feat in features.items():
        if img_id not in descriptions:
            continue
        X1, X2, y = create_sequences(tokenizer, max_len, descriptions[img_id], feat)
        for i in range(len(X1)):
            yield X1[i], X2[i], y[i]

def build_dataset(features, descriptions, tokenizer, max_len, batch_size=32):
    def gen():
        for img_id, feat in features.items():
            if img_id not in descriptions:
                continue
            X1, X2, y = create_sequences(tokenizer, max_len, descriptions[img_id], feat)
            for i in range(len(X1)):
                # Yield inputs tuple and output separately
                yield (X1[i], X2[i]), y[i]

    sample = next(gen())
    output_types = ((tf.float32, tf.int32), tf.float32)
    output_shapes = ((sample[0].shape, sample[1].shape), sample[2].shape)

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_types=output_types,
        output_shapes=output_shapes
    )
    dataset = dataset.shuffle(1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


In [9]:
from pathlib import Path
import json, pickle, numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ---------- paths ----------
data_dir = Path(".")
train_feat_path = data_dir / "train_image_features_tfhub.pkl"
val_feat_path   = data_dir / "val_image_features_tfhub.pkl"
train_cap_path  = data_dir / "processed_captions.json"
val_cap_path    = data_dir / "processed_val_captions.json"
tokenizer_path  = data_dir / "tokenizer.pkl"

# ---------- load and fix val captions ----------
with open(val_cap_path) as fh:
    val_caps = json.load(fh)
print("Converting val_caps from list to dict...")
if isinstance(val_caps, list):
    val_caps = {item["image_id"]: item["captions"] for item in val_caps}

# ---------- load and fix train captions ----------
with open(train_cap_path) as fh:
    train_caps = json.load(fh)
print("Converting train_caps from list to dict...")
if isinstance(train_caps, list):
    train_caps = {item["image_id"]: item["captions"] for item in train_caps}

# ---------- merge captions ----------
image_captions_dict = {**train_caps, **val_caps}

# ---------- load image features ----------
with open(train_feat_path, "rb") as fh:
    train_feats = pickle.load(fh)
with open(val_feat_path, "rb") as fh:
    val_feats = pickle.load(fh)

# ---------- merge image features ----------
image_features_dict = {**train_feats, **val_feats}

# ---------- reduce dataset size ----------
MAX_IMAGES = 1000  # adjust based on available RAM
subset_keys = [k for k in image_captions_dict if k in image_features_dict][:MAX_IMAGES]
image_captions_dict = {k: image_captions_dict[k] for k in subset_keys}
image_features_dict = {k: image_features_dict[k] for k in subset_keys}

# ---------- load tokenizer ----------
with open(tokenizer_path, "rb") as fh:
    tokenizer = pickle.load(fh)

# ---------- compute max_len ----------
all_caps = sum(image_captions_dict.values(), [])
max_len = max(len(c.split()) for c in all_caps)

print(f"Loaded {len(image_features_dict)} images, {len(all_caps)} captions, max_len={max_len}")
print(f"Average captions per image: {len(all_caps)/len(image_captions_dict):.2f}")

# ---------- build_dataset function ----------
# This is a sample build_dataset function using create_sequences()
def build_dataset(features, descriptions, tokenizer, max_len, batch_size=32):
    X1_list, X2_list, y_list = [], [], []
    
    for img_id, caps in descriptions.items():
        photo_feat = features[img_id]
        X1, X2, y = create_sequences(tokenizer, max_len, caps, photo_feat)
        X1_list.append(X1)
        X2_list.append(X2)
        y_list.append(y)

    # Concatenate all arrays
    X1_all = np.concatenate(X1_list, axis=0)
    X2_all = np.concatenate(X2_list, axis=0)
    y_all = np.concatenate(y_list, axis=0)

    # Create a tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices(((X1_all, X2_all), y_all))
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

# ---------- create_sequences function ----------
def create_sequences(tokenizer, max_len, captions, photo_feat):
    X1, X2, y = [], [], []
    vocab_size = len(tokenizer.word_index) + 1
    photo_feat = np.array(photo_feat, dtype=np.float32)

    for caption in captions:
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
            in_seq, out_word = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_len)[0]

            out_seq = np.zeros(vocab_size, dtype=np.float32)
            out_seq[out_word] = 1.0

            X1.append(photo_feat)
            X2.append(in_seq)
            y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y)

# ---------- build the dataset ----------
dataset = build_dataset(
    features=image_features_dict,
    descriptions=image_captions_dict,
    tokenizer=tokenizer,
    max_len=max_len,
    batch_size=32
)


Converting val_caps from list to dict...
Converting train_caps from list to dict...
Loaded 1000 images, 5007 captions, max_len=37
Average captions per image: 5.01


## Step 5: Image Caption Training Model

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

# Hyperparameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256
units = 256

# Image feature input branch
inputs1 = Input(shape=(1280,))  # shape matches your extracted features
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# Caption input branch
inputs2 = Input(shape=(max_len,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# Decoder (merge)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# Define and compile model
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 37)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 1280)]       0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 37, 256)      7574272     ['input_7[0][0]']                
                                                                                                  
 dropout_10 (Dropout)           (None, 1280)         0           ['input_6[0][0]']                
                                                                                            

In [42]:
LIMIT = 35000
subset_dataset = dataset.take(LIMIT // 32)  # batch size 32

history = model.fit(
    subset_dataset,
    epochs=10,
    verbose=1
)

Epoch 1/10
1093/1093 [==============================] - 474s 430ms/step - loss: 5.0566
Epoch 2/10
1093/1093 [==============================] - 475s 434ms/step - loss: 3.9113
Epoch 3/10
1093/1093 [==============================] - 481s 440ms/step - loss: 3.5153
Epoch 4/10
1093/1093 [==============================] - 486s 445ms/step - loss: 3.2260
Epoch 5/10
1093/1093 [==============================] - 488s 446ms/step - loss: 2.9781
Epoch 6/10
1093/1093 [==============================] - 486s 445ms/step - loss: 2.7775
Epoch 7/10
1093/1093 [==============================] - 486s 445ms/step - loss: 2.5782
Epoch 8/10
1093/1093 [==============================] - 490s 448ms/step - loss: 2.4122
Epoch 9/10
1093/1093 [==============================] - 486s 445ms/step - loss: 2.2592
Epoch 10/10
1093/1093 [==============================] - 474s 433ms/step - loss: 2.1208


In [43]:
model.save("saved_model_dir", save_format="tf")

INFO:tensorflow:Assets written to: saved_model_dir\assets


INFO:tensorflow:Assets written to: saved_model_dir\assets


In [44]:
from tensorflow.keras.models import load_model
model = load_model("saved_model_dir")

In [45]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the trained model (already done)
# Load the tokenizer and max_len (already done)

# Use MobileNetV2 (same as during training)
mobilenet_v2_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(mobilenet_v2_url, input_shape=(224, 224, 3), trainable=False)

# Function to extract features for test image
def extract_features(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = feature_extractor(img_array)
    return tf.squeeze(features).numpy()

# Function to generate caption
def generate_caption(model, tokenizer, photo_features, max_len):
    input_text = 'startseq'
    for i in range(max_len):
        sequence = tokenizer.texts_to_sequences([input_text])[0]
        sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_len)
        yhat = model.predict([np.expand_dims(photo_features, axis=0), sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat, None)
        if word is None:
            break
        input_text += ' ' + word
        if word == 'endseq':
            break
    return input_text.replace('startseq', '').replace('endseq', '').strip()


In [52]:
def generate_caption(model, tokenizer, photo_features, max_len):
    input_text = 'startseq'
    for _ in range(max_len):
        sequence = tokenizer.texts_to_sequences([input_text])[0]
        sequence = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_len)

        photo_features_exp = np.expand_dims(photo_features, axis=0)  # add batch dim

        yhat_probs = model.predict([photo_features_exp, sequence], verbose=0)
        yhat = np.argmax(yhat_probs, axis=-1)[0]

        word = tokenizer.index_word.get(yhat, None)
        if word is None:
            break
        if word == '<unk>':
            continue  # skip unknown words during generation

        input_text += ' ' + word  # append predicted word

        if word == 'endseq':
            break

    caption = input_text.split()[1:-1]  # remove 'startseq' and 'endseq'
    return ' '.join(caption)


In [53]:
image_path = r"D:\Image Captioning and Segmentation\val2017\000000000285.jpg"
photo_features = extract_features(image_path)  # returns shape (1280,)
caption = generate_caption(model, tokenizer, photo_features, max_len)
print("Generated Caption:", caption)

Generated Caption: a man in a kitchen with a huge black floor


## BLEU Evaluation

In [55]:
import pickle
import spacy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Load spacy tokenizer (English)
nlp = spacy.load("en_core_web_sm")

def spacy_tokenize(text):
    return [token.text.lower() for token in nlp(text)]

# Load pre-extracted validation image features
with open("val_image_features_tfhub.pkl", "rb") as f:
    val_image_features = pickle.load(f)

# Assume you have your val_data JSON loaded as well (list of dicts)
# with keys like 'image_id' and 'captions' (list of references)
import json
with open("processed_val_captions.json", "r") as f:
    val_data = json.load(f)

smooth_fn = SmoothingFunction().method1

bleu_scores = []

for entry in val_data:
    image_id = str(entry["image_id"]).zfill(12)
    references = [spacy_tokenize(c) for c in entry["captions"]]

    photo_feat = val_image_features[image_id]
    generated_caption = generate_caption(model, tokenizer, photo_feat, max_len)
    tokenized_candidate = spacy_tokenize(generated_caption)

    score = sentence_bleu(
        references, tokenized_candidate, smoothing_function=smooth_fn
    )
    bleu_scores.append(score)

average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score on validation set: {average_bleu:.4f}")

Average BLEU score on validation set: 0.0703


## Step 6: Image Segmentation

In [62]:
import os
from ultralytics import YOLO
from PIL import Image
from tqdm import tqdm

# Paths
val_image_dir = r"D:\Image Captioning and Segmentation\val2017"
segmented_output_dir = r"D:\Image Captioning and Segmentation\val2017_segmented"
os.makedirs(segmented_output_dir, exist_ok=True)

# Load segmentation model
model = YOLO("yolov8n-seg.pt")  # Ensure this is a segmentation model

# Device: 'cpu' is safer, switch to 'cuda' if it's stable for you
device_type = 'cpu'  # or 'cuda'

# Batch processing
image_files = [f for f in os.listdir(val_image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

for img_file in tqdm(image_files, desc="Segmenting Images"):
    image_path = os.path.join(val_image_dir, img_file)
    
    # Run segmentation
    try:
        results = model(image_path, device=device_type)
        segmented_img = Image.fromarray(results[0].plot())

        # Save segmented image
        output_path = os.path.join(segmented_output_dir, img_file)
        segmented_img.save(output_path)
    except Exception as e:
        print(f"Error processing {img_file}: {e}")


Segmenting Images:   0%|          | 0/5000 [00:00<?, ?it/s]


image 1/1 D:\Image Captioning and Segmentation\val2017\000000000139.jpg: 448x640 1 person, 3 chairs, 2 potted plants, 1 dining table, 2 tvs, 1 refrigerator, 1 clock, 1 vase, 111.3ms
Speed: 2.0ms preprocess, 111.3ms inference, 14.4ms postprocess per image at shape (1, 3, 448, 640)


Segmenting Images:   0%|          | 1/5000 [00:00<21:29,  3.88it/s]


image 1/1 D:\Image Captioning and Segmentation\val2017\000000000285.jpg: 640x608 1 bear, 85.6ms
Speed: 1.8ms preprocess, 85.6ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 608)


Segmenting Images:   0%|          | 2/5000 [00:00<15:19,  5.44it/s]

...

Segmenting Images: 100%|█████████▉| 4998/5000 [11:46<00:00,  6.60it/s]


image 1/1 D:\Image Captioning and Segmentation\val2017\000000581615.jpg: 640x480 1 toilet, 65.8ms
Speed: 2.7ms preprocess, 65.8ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)


Segmenting Images: 100%|█████████▉| 4999/5000 [11:46<00:00,  7.00it/s]


image 1/1 D:\Image Captioning and Segmentation\val2017\000000581781.jpg: 480x640 16 bananas, 80.1ms
Speed: 5.0ms preprocess, 80.1ms inference, 15.7ms postprocess per image at shape (1, 3, 480, 640)


Segmenting Images: 100%|██████████| 5000/5000 [11:47<00:00,  7.07it/s]


In [97]:
annotations_file = r"D:\Image Captioning and Segmentation\annotations\instances_train2017.json"
images_dir = r"D:\Image Captioning and Segmentation\train2017"
labels_dir = r"path_to_save_labels/train2017_labels"

convert_coco_to_yolo(annotations_file, images_dir, labels_dir)

Conversion complete. Labels saved in path_to_save_labels/train2017_labels


In [3]:
from ultralytics import YOLO
from PIL import Image

model = YOLO("yolov8n-seg.pt")
img_path = r"D:\Image Captioning and Segmentation\val2017\000000000802.jpg"


results = model(img_path, device='cpu')
result_img = Image.fromarray(results[0].plot())
result_img.show()


image 1/1 D:\Image Captioning and Segmentation\val2017\000000000802.jpg: 640x448 1 oven, 1 refrigerator, 83.0ms
Speed: 1.2ms preprocess, 83.0ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 448)
